In [1]:
import xarray as xr
xr.set_options(display_style='html')
import intake
import cftime
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd
import dask
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import cartopy as cy
#from functions import trend
import os as os
#import pymannkendall as mk
%matplotlib inline

In [2]:
! pip install pymannkendall
import pymannkendall as mk

  Using cached pymannkendall-1.4.2-py3-none-any.whl (12 kB)


In [3]:
'''
urls = ['http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/AERmon/co2/gn/v20191108/co2_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', 
 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/AERmon/emibc/gn/v20191108/emibc_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', 
 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/AERmon/emibvoc/gn/v20191108/emibvoc_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', 
 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/AERmon/emidms/gn/v20191108/emidms_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', 
 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/AERmon/emidust/gn/v20191108/emidust_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', 
 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/AERmon/emiisop/gn/v20191108/emiisop_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', 
 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/AERmon/emioa/gn/v20191108/emioa_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', 
 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/AERmon/emiso2/gn/v20191108/emiso2_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', 
 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/AERmon/emiso4/gn/v20191108/emiso4_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', 
 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/AERmon/emiss/gn/v20191108/emiss_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', 
 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/AERmon/emivoc/gn/v20191108/emivoc_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', 
 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/AERmon/so2/gn/v20191108/so2_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', 
 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/Amon/prc/gn/v20191108/prc_Amon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', 
 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/Amon/pr/gn/v20191108/pr_Amon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', 
 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/Amon/ts/gn/v20191108/ts_Amon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', 
 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-LM/abrupt-4xCO2/r1i1p1f1/AERmon/emidms/gn/v20210118/emidms_AERmon_NorESM2-LM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', 
 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/AERmon/ccn/gn/v20191108/ccn_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', 
 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/AERmon/cdnc/gn/v20191108/cdnc_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', 
 #'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/AERmon/dms/gn/v20191108/dms_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012'
       ]

# wget all files in the list
for url in urls:
    os.system('wget ' + url)
'''

"\nurls = ['http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/AERmon/co2/gn/v20191108/co2_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', \n 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/AERmon/emibc/gn/v20191108/emibc_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', \n 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/AERmon/emibvoc/gn/v20191108/emibvoc_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', \n 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/AERmon/emidms/gn/v20191108/emidms_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc', \n 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/abrupt-4xCO2/r1i1p1f1/AERmon/emidust/gn/v20191

In [4]:
#url = 'http://noresg.nird.sigma2.no/thredds/fileServer/esg_dataroot/cmor/CMIP6/CMIP/NCC/NorESM2-MM/piControl/r1i1p1f1/AERmon/conccn/gn/v20191108/conccn_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc'
#os.system('wget '+ url)

In [5]:
# Read in multiple piControl files:

list_of_files10_piControl = [
    '../../Data/ccn_AERmon_NorESM2-MM_piControl_r1i1p1f1_gn_120001-120912.nc',
    '../../Data/cdnc_AERmon_NorESM2-MM_piControl_r1i1p1f1_gn_120001-120912.nc',
    '../../Data/co2_AERmon_NorESM2-MM_piControl_r1i1p1f1_gn_120001-120912.nc',
    #'../../Data/dms_AERmon_NorESM2-MM_piControl_r1i1p1f1_gn_120001-120912.nc',
    '../../Data/emibc_AERmon_NorESM2-MM_piControl_r1i1p1f1_gn_120001-120912.nc',
    '../../Data/emibvoc_AERmon_NorESM2-MM_piControl_r1i1p1f1_gn_120001-120912.nc',
    #'../../Data/emidms_AERmon_NorESM2-MM_piControl_r1i1p1f1_gn_120001-120912.nc',
    '../../Data/emidust_AERmon_NorESM2-MM_piControl_r1i1p1f1_gn_120001-120912.nc',
    '../../Data/emioa_AERmon_NorESM2-MM_piControl_r1i1p1f1_gn_120001-120912.nc',
    '../../Data/emiso2_AERmon_NorESM2-MM_piControl_r1i1p1f1_gn_120001-120912.nc',
    '../../Data/emiso4_AERmon_NorESM2-MM_piControl_r1i1p1f1_gn_120001-120912.nc',
    '../../Data/emiss_AERmon_NorESM2-MM_piControl_r1i1p1f1_gn_120001-120912.nc',
    '../../Data/pr_Amon_NorESM2-MM_piControl_r1i1p1f1_gn_120001-120912.nc',
    '../../Data/prc_Amon_NorESM2-MM_piControl_r1i1p1f1_gn_120001-120912.nc',
    '../../Data/ts_Amon_NorESM2-MM_piControl_r1i1p1f1_gn_120001-120912.nc',
    '../../Data/so2_AERmon_NorESM2-MM_piControl_r1i1p1f1_gn_120001-120912.nc',
    '../../Data/emivoc_AERmon_NorESM2-MM_piControl_r1i1p1f1_gn_120001-120912.nc',
    '../../Data/emiisop_AERmon_NorESM2-MM_piControl_r1i1p1f1_gn_120001-120912.nc'
]


# missing so2, emiisop, emivoc
ds_piControl10 = xr.open_mfdataset(list_of_files10_piControl, combine='by_coords', compat='override', use_cftime=True)#concat_dim='time',combine='by_coords')
ds_piControl10


<xarray.Dataset>
Dimensions:    (time: 120, bnds: 2, lev: 32, lat: 192, lon: 288)
Coordinates:
  * time       (time) object 1200-01-16 12:00:00 ... 1209-12-16 12:00:00
  * lev        (lev) float64 0.9926 0.9763 0.9575 ... 0.01436 0.007595 0.003643
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Dimensions without coordinates: bnds
Data variables: (12/26)
    time_bnds  (time, bnds) object dask.array<chunksize=(120, 2), meta=np.ndarray>
    lev_bnds   (lev, bnds) float64 dask.array<chunksize=(32, 2), meta=np.ndarray>
    p0         float64 ...
    a          (lev) float64 dask.array<chunksize=(32,), meta=np.ndarray>
    b          (lev) float64 dask.array<chunksize=(32,), meta=np.ndarray>
    ps         (time, lat, lon) float32 dask.array<chunksize=(120, 192, 288), meta=np.ndarray>
    ...         ...
    emiso4     (time, lat, lon) float32 dask.array<chunksize=(120, 192, 288), meta=np.ndarray>
    emiss      (time, lat, lon) float32 dask.array<chunksize=(120, 192, 288), meta=np.ndarray>
    emivoc     (time, lat, lon) float32 dask.array<chunksize=(120, 192, 288), meta=np.ndarray>
    pr         (time, lat, lon) float32 dask.array<chunksize=(120, 192, 288), meta=np.ndarray>
    prc        (time, lat, lon) float32 dask.array<chunksize=(120, 192, 288), meta=np.ndarray>
    ts         (time, lat, lon) float32 dask.array<chunksize=(120, 192, 288), meta=np.ndarray>
Attributes: (12/49)
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               CMIP
    branch_method:             Hybrid-restart from year 1200-01-01 of piContr...
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     438000.0
    ...                        ...
    title:                     NorESM2-MM output prepared for CMIP6
    variable_id:               cdnc
    variant_label:             r1i1p1f1
    license:                   CMIP6 model data produced by NCC is licensed u...
    cmor_version:              3.5.0
    tracking_id:               hdl:21.14100/9514c407-05be-44d4-9aee-cdf51d923312

In [6]:
ds_piControl10 = ds_piControl10.assign_coords(lon=(((ds_piControl10.lon + 180) % 360) - 180)).sortby('lon')

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/srv/conda/envs/note

In [7]:

ds_piControl20 = xr.open_mfdataset('../../Data/NorESM2-MM_piControl_r1i1p1f1_gn_121001-121912.nc', use_cftime=True)#concat_dim='time',combine='by_coords')
ds_piControl30 = xr.open_mfdataset('../../Data/NorESM2-MM_piControl_r1i1p1f1_gn_122001-122912.nc', use_cftime=True)#concat_dim='time',combine='by_coords')


In [8]:
list_of_files20 = [
    #'../../Data/all_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_000101-001012.nc',
    '../../Data/ccn_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_001101-002012.nc',
    '../../Data/pr_Amon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_001101-002012.nc',
    '../../Data/cdnc_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_001101-002012.nc',
    '../../Data/co2_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_001101-002012.nc',
    '../../Data/emibc_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_001101-002012.nc',
    '../../Data/emibvoc_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_001101-002012.nc',
    #'../../Tjaernoe2022-group7/ingrblu/emidms_AERmon_NorESM2-LM_abrupt-4xCO2_r1i1p1f1_gn_001101-002012.nc',
    '../../Data/emidust_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_001101-002012.nc',
    '../../Data/emiisop_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_001101-002012.nc',
    '../../Data/emioa_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_001101-002012.nc',
    '../../Data/emiso2_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_001101-002012.nc',
    '../../Data/emiso4_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_001101-002012.nc',
    '../../Data/emiss_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_001101-002012.nc',
    '../../Data/emivoc_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_001101-002012.nc',
    '../../Data/prc_Amon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_001101-002012.nc',
    '../../Data/so2_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_001101-002012.nc',
    '../../Data/ts_Amon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_001101-002012.nc']

ds_4xco2_20 = xr.open_mfdataset(list_of_files20, combine='by_coords', compat='override', use_cftime=True)#concat_dim='time',combine='by_coords')
ds_4xco2_20

<xarray.Dataset>
Dimensions:    (time: 120, bnds: 2, lev: 32, lat: 192, lon: 288)
Coordinates:
  * time       (time) object 0011-01-16 12:00:00 ... 0020-12-16 12:00:00
  * lev        (lev) float64 0.9926 0.9763 0.9575 ... 0.01436 0.007595 0.003643
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Dimensions without coordinates: bnds
Data variables: (12/26)
    time_bnds  (time, bnds) object dask.array<chunksize=(120, 2), meta=np.ndarray>
    lev_bnds   (lev, bnds) float64 dask.array<chunksize=(32, 2), meta=np.ndarray>
    p0         float64 ...
    a          (lev) float64 dask.array<chunksize=(32,), meta=np.ndarray>
    b          (lev) float64 dask.array<chunksize=(32,), meta=np.ndarray>
    ps         (time, lat, lon) float32 dask.array<chunksize=(120, 192, 288), meta=np.ndarray>
    ...         ...
    emiso4     (time, lat, lon) float32 dask.array<chunksize=(120, 192, 288), meta=np.ndarray>
    emiss      (time, lat, lon) float32 dask.array<chunksize=(120, 192, 288), meta=np.ndarray>
    emivoc     (time, lat, lon) float32 dask.array<chunksize=(120, 192, 288), meta=np.ndarray>
    pr         (time, lat, lon) float32 dask.array<chunksize=(120, 192, 288), meta=np.ndarray>
    prc        (time, lat, lon) float32 dask.array<chunksize=(120, 192, 288), meta=np.ndarray>
    ts         (time, lat, lon) float32 dask.array<chunksize=(120, 192, 288), meta=np.ndarray>
Attributes: (12/49)
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               CMIP
    branch_method:             Hybrid-restart from year 1200-01-01 of piControl
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     438000.0
    ...                        ...
    title:                     NorESM2-MM output prepared for CMIP6
    variable_id:               cdnc
    variant_label:             r1i1p1f1
    license:                   CMIP6 model data produced by NCC is licensed u...
    cmor_version:              3.5.0
    tracking_id:               hdl:21.14100/e6a80728-03d9-4a24-87b1-67a45044fdbd

In [9]:
list_of_files30 = [
    #'../../Data/all_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_000101-001012.nc',
    '../../Data/ccn_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc',
    '../../Data/pr_Amon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc',
    '../../Data/cdnc_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc',
    '../../Data/co2_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc',
    '../../Data/emibc_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc',
    '../../Data/emibvoc_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc',
    #'../../Tjaernoe2022-group7/ingrblu/emidms_AERmon_NorESM2-LM_abrupt-4xCO2_r1i1p1f1_gn_001101-002012.nc',
    '../../Data/emidust_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc',
    '../../Data/emiisop_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc',
    '../../Data/emioa_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc',
    '../../Data/emiso2_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc',
    '../../Data/emiso4_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc',
    '../../Data/emiss_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc',
    '../../Data/emivoc_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc',
    '../../Data/prc_Amon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc',
    '../../Data/so2_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc',
    '../../Data/ts_Amon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc']

ds_4xco2_30 = xr.open_mfdataset(list_of_files30, combine='by_coords', compat='override', use_cftime=True)#concat_dim='time',combine='by_coords')
ds_4xco2_30

<xarray.Dataset>
Dimensions:    (time: 120, bnds: 2, lev: 32, lat: 192, lon: 288)
Coordinates:
  * time       (time) object 0021-01-16 12:00:00 ... 0030-12-16 12:00:00
  * lev        (lev) float64 0.9926 0.9763 0.9575 ... 0.01436 0.007595 0.003643
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Dimensions without coordinates: bnds
Data variables: (12/26)
    time_bnds  (time, bnds) object dask.array<chunksize=(120, 2), meta=np.ndarray>
    lev_bnds   (lev, bnds) float64 dask.array<chunksize=(32, 2), meta=np.ndarray>
    p0         float64 ...
    a          (lev) float64 dask.array<chunksize=(32,), meta=np.ndarray>
    b          (lev) float64 dask.array<chunksize=(32,), meta=np.ndarray>
    ps         (time, lat, lon) float32 dask.array<chunksize=(120, 192, 288), meta=np.ndarray>
    ...         ...
    emiso4     (time, lat, lon) float32 dask.array<chunksize=(120, 192, 288), meta=np.ndarray>
    emiss      (time, lat, lon) float32 dask.array<chunksize=(120, 192, 288), meta=np.ndarray>
    emivoc     (time, lat, lon) float32 dask.array<chunksize=(120, 192, 288), meta=np.ndarray>
    pr         (time, lat, lon) float32 dask.array<chunksize=(120, 192, 288), meta=np.ndarray>
    prc        (time, lat, lon) float32 dask.array<chunksize=(120, 192, 288), meta=np.ndarray>
    ts         (time, lat, lon) float32 dask.array<chunksize=(120, 192, 288), meta=np.ndarray>
Attributes: (12/49)
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               CMIP
    branch_method:             Hybrid-restart from year 1200-01-01 of piControl
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     438000.0
    ...                        ...
    title:                     NorESM2-MM output prepared for CMIP6
    variable_id:               cdnc
    variant_label:             r1i1p1f1
    license:                   CMIP6 model data produced by NCC is licensed u...
    cmor_version:              3.5.0
    tracking_id:               hdl:21.14100/02801f3b-389d-47a2-9e75-e10706170c71

In [10]:
ds_4xco2_10 = xr.open_mfdataset('../../Data/all_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_000101-001012.nc', use_cftime=True)#concat_dim='time',combine='by_coords')

In [11]:

time20 = ds_4xco2_20.time.values
time30 = ds_4xco2_30.time.values

#time20_piControl = ds_piControl20.time.values
##time30_piControl = ds_piControl_30.time.values



In [12]:
def convert(ds, list_of_files, time):

    levels = ds_4xco2_10.lev.values
    lat = ds_4xco2_10.lat.values
    lon = ds_4xco2_10.lon.values

        

    ds_all = xr.Dataset({'time': time, 'lev': levels, 'lat': lat,'lon': lon})
    for path in list_of_files:
            ds = xr.open_dataset(path)
            for var in ds.data_vars:
                    a = str(var)
            try:
                    ds_all[a] = (['time', 'lat', 'lon'], ds[a].values)
            except:
                    print(path)
                    ds_all[a] = (['time', 'lev', 'lat', 'lon'], ds[a].values)

    ds_all = ds_all.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    return ds_all



In [13]:
ds_20 = convert(ds_4xco2_20, list_of_files20, time20)

../../Data/cdnc_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_001101-002012.nc
../../Data/co2_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_001101-002012.nc
../../Data/so2_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_001101-002012.nc


In [14]:
ds_30 = convert(ds_4xco2_30, list_of_files30, time30)

../../Data/cdnc_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc
../../Data/co2_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc
../../Data/so2_AERmon_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_gn_002101-003012.nc


In [15]:
    
    
var_list = ['pr', 'ccn', 'cdnc', 'co2', 'emibc', 'emibvoc', 'emidust', 'emiisop', 'emioa', 'emiso2', 'emiso4', 'emiss', 'emivoc', 'prc', 'so2', 'ts']
#var_list = ds_20.data_vars

ds_all = xr.concat([ds_4xco2_10[var_list], ds_20[var_list], ds_30[var_list]], dim="time")


In [16]:
# making the three piControl datasets concatable 

variables = ["time_bnds", "lev_bnds", "p0", "a", "b", "ps", "a_bnds", "b_bnds", "lat_bnds", "lon_bnds"]
ds_piControl10 = ds_piControl10.drop_vars(variables)

variables = ["dms", "emidms"]
ds_piControl20 = ds_piControl20.drop_vars(variables)
ds_piControl30 = ds_piControl30.drop_vars(variables)

In [17]:
ds_all_piControl = xr.concat([ds_piControl10[var_list], ds_piControl20[var_list], ds_piControl30[var_list]], dim='time')



In [18]:
#ds_4xco2_10.to_netcdf('../../Data/all_NorESM2-MM_abrupt-4xCO2_r1i1p1f1_000101-002012.nc').append(ds_4xco2_20)

In [19]:
# function to remove dimensions with only one value
def remove_singledim(ds):
    for dim in ds.dims:
        if len(ds[dim]) == 1:
            ds = ds.squeeze(dim)
    return ds

In [20]:
def annual(ds):
    return ds.resample(time="Y").mean()


ds_4xco2_NorESM_y = annual(ds_all)
ds_piControl_NorESM_y = annual(ds_all_piControl)



/srv/conda/envs/notebook/lib/python3.9/site-packages/dask/array/routines.py:1995: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return a[(slice(None),) * axis + (indices,)]


In [21]:
def fix_units(ds):
    ds['pr'] = ds['pr']*86400 # mm/year
    ds['pr'] = ds['pr'].assign_attrs(units='mm/year')
    ds['ts'] = ds['ts']-273.15 # C
    ds['ts'] = ds['ts'].assign_attrs(units='C$^\circ$')
    return ds




In [22]:
ds_4xco2_NorESM_y = fix_units(ds_4xco2_NorESM_y)
ds_piControl_NorESM_y = fix_units(ds_piControl_NorESM_y)

In [23]:
#ds_trend_4xco2 = mk_trend(ds_4xco2_NorESM_y.pr)
#ds_pval_4xco2 = mk_pval(ds_4xco2_NorESM_y.pr)


In [24]:
import numpy as np
import pymannkendall as mk

def mk_slope(y):
    '''
    function that returns the slope of a linear regression using numpy polyfit
    '''
    return mk.original_test(y)[7]
    
def mk_pval(y):
    '''
    function that returns the slope of a linear regression using numpy polyfit
    '''
    # use xr.apply_ufunc to calculate the trend of the variable on each grid point
    return mk.original_test(y)[2]

def mk_trend(ds, dim="time"):
    '''
    Calculate the trend of all variables in a dataset using numpy polyfit for all grid points.
    Returns a dataset with the same variables as the input dataset
    '''
    ds_chunk = ds.chunk({dim: -1})
    trend = xr.apply_ufunc(
        mk_slope,
        ds_chunk,
        vectorize=True,
        input_core_dims=[[dim]],
        output_core_dims=[[]],
        output_dtypes=[np.float64],
        dask="parallelized",
    )
    return trend

def mk_pval(ds, dim="time"):
    '''
    Calculate the trend of all variables in a dataset using numpy polyfit for all grid points.
    Returns a dataset with the same variables as the input dataset
    '''
    ds_chunk = ds.chunk({dim: -1})
    pval = xr.apply_ufunc(
        mk_pval,
        ds_chunk,
        vectorize=True,
        input_core_dims=[[dim]],
        output_core_dims=[[]],
        output_dtypes=[np.float64],
        dask="parallelized",
    )
    return pval



In [25]:
ds_trend_4xco2 = mk_trend(ds_4xco2_NorESM_y.chunk({"lat": 20, "lon": 20, "time": -1}))
ds_trend_piControl = mk_trend(ds_piControl_NorESM_y.chunk({"lat": 20, "lon": 20, "time": -1}))
ds_pval_4xco2 = mk_pval(ds_4xco2_NorESM_y.chunk({"lat": 20, "lon": 20, "time": -1}))


In [26]:
def trend(ds, variable):#, season):
    
    # create a dataset with same lon and lat as the precipitation dataset
    ds_trend = xr.Dataset({'lat': ds.lat,'lon': ds.lon})
    # add ds_trend_DJF to ds_NorESM2_MM_precip_season_trend
    # ds_season = ds.groupby('time.season').mean(keep_attrs=True) eks
    ds_trend['mk_trend'] = (['lat', 'lon'], np.zeros((ds.lat.shape[0], ds.lon.shape[0]))*np.nan)
    ds_trend['mk_intercept'] = (['lat', 'lon'], np.zeros((ds.lat.shape[0], ds.lon.shape[0]))*np.nan)
    ds_trend['mk_p_val'] = (['lat', 'lon'], np.zeros((ds.lat.shape[0], ds.lon.shape[0]))*np.nan)  
    array_var = ds[variable].values
    for ilat in range(array_var.shape[1]):
        for ilon in range(array_var.shape[2]):
            ts = array_var[:, ilat, ilon]
            results = mk.original_test(ts)
            ds_trend['mk_trend'][ilat, ilon] = results[7]
            ds_trend['mk_intercept'][ilat, ilon] = results[8]
            ds_trend['mk_p_val'][ilat, ilon] = results[2]
    return ds_trend



In [27]:
ccn_piControl_trend_NorESM_y = trend(ds_piControl_NorESM_y, 'ccn') 

/srv/conda/envs/notebook/lib/python3.9/site-packages/flox/aggregate_flox.py:105: RuntimeWarning: invalid value encountered in divide
  out /= nanlen(group_idx, array, size=size, axis=axis, fill_value=0)


ZeroDivisionError: float division by zero

In [ ]:
ts_piControl_trend_NorESM_y = trend(ds_piControl_NorESM_y, 'ts') 

In [ ]:
ts_piControl_trend_NorESM_y['mk_trend']


In [ ]:
ts_4xco2_trend_NorESM_y = trend(ds_4xco2_NorESM_y, 'ts') #, 'JJA')

In [ ]:
ccn_4xco2_trend_NorESM_y = trend(ds_4xco2_NorESM_y, 'ccn') #, 'JJA')

In [ ]:
pr_4xco2_trend_NorESM_y = trend(ds_4xco2_NorESM_y, 'pr') #, 'JJA')

In [ ]:
pr_piControl_trend_NorESM_y = trend(ds_piControl_NorESM_y, 'pr') 

In [ ]:

def dot_signif(ds, ax):
    for ilat, lat in enumerate(ds.lat):
        for ilon, lon in enumerate(ds.lon):
            if ds.mk_p_val.isel(lat=ilat, lon=ilon) < 0.05:
                if (ilon % 3 == 0) and (ilat % 3 == 0):
                    ax.plot(lon, lat, marker='o', color='k', transform=ccrs.PlateCarree(), markersize=1)


In [ ]:
'''
def dot_signif(ds, var, ax):
    for ilat, lat in enumerate(ds.lat):
        for ilon, lon in enumerate(ds.lon):
            if ds[var].isel(lat=ilat, lon=ilon) < 0.05:
                if (ilon % 3 == 0) and (ilat % 3 == 0):
                    ax.plot(lon, lat, marker='o', color='k', transform=ccrs.PlateCarree(), markersize=1)
'''

In [ ]:
def plot_map_trend(ds, var, title, levels=6, extent=[-180,180,90,50], vmin=-0.12, vmax=0.12):
    '''
        Plots the trend on a map from the model data
        
    Args:
    -----
        var    [DataArray]   :   An xarray contining data about the conditions 
        levels [int]         :   Levels in the plot
        extent [list]        :   A list of the extent of the plot
    '''
    
    
    sat_proj = ccrs.NorthPolarStereo()
    fig, ax = plt.subplots(figsize=(7,7),subplot_kw={'projection':sat_proj})
    ds[var].plot(facecolor="gray",
        ax = ax,
        cbar_kwargs={
            'orientation':'vertical',
            'shrink':.8
            },                 
        transform=ccrs.PlateCarree(),
        #levels=levels,
        vmin=vmin,
        vmax=vmax, 
        cmap="RdBu"
        #robust=True
        )


    #dot_signif(ds, ax)
    
    #cbar.set_clim(min, max)
    ax.set_extent(extent, ccrs.PlateCarree())
    ax.gridlines(draw_labels=True)
    ax.coastlines()
    ax.set_title(title, fontweight='bold', fontsize=15)
    fig.tight_layout()

In [ ]:
fig = plt.figure(figsize=(12, 5))
ax = plt.subplot(111, projection=ccrs.Robinson())

pr_piControl_trend_NorESM_y.mk_trend.plot(facecolor="gray", transform=ccrs.PlateCarree(), ax=ax)
# coastlines
ax.coastlines()
ax.set_title('Global annual trend in precipitation (piControl)', fontweight='bold', fontsize=15)

dot_signif(pr_piControl_trend_NorESM_y, ax)

In [ ]:
fig = plt.figure(figsize=(12, 5))
ax = plt.subplot(111, projection=ccrs.Robinson())

ts_4xco2_trend_NorESM_y.mk_trend.plot(facecolor="gray", transform=ccrs.PlateCarree(), ax=ax)
# coastlines
ax.coastlines()
ax.set_title('Global annual trend in temperature (4xco2)', fontweight='bold', fontsize=15)

dot_signif(ts_4xco2_trend_NorESM_y, ax)

In [ ]:
fig = plt.figure(figsize=(12, 5))
ax = plt.subplot(111, projection=ccrs.Robinson())

ds_trend_4xco2['pr'].plot(facecolor="gray", transform=ccrs.PlateCarree(), ax=ax)
# coastlines
ax.coastlines()
ax.set_title('Global annual trend in precipitation (4xco2)', fontweight='bold', fontsize=15)

dot_signif(pr_4xco2_trend_NorESM_y, ax)


In [ ]:
fig = plt.figure(figsize=(12, 5))
ax = plt.subplot(111, projection=ccrs.Robinson())

ds_trend_piControl['ts'].plot(facecolor="gray", transform=ccrs.PlateCarree(), ax=ax)
# coastlines
ax.coastlines()
ax.set_title('Global annual trend in temp (piControl)', fontweight='bold', fontsize=15)

dot_signif(ts_piControl_trend_NorESM_y, ax)


In [ ]:
plot_map_trend(ds_trend_4xco2, 'pr', 'Trend in annual precipitation (abrupt-4xCO2)')  

In [ ]:
plot_map_trend(ds_trend_piControl, 'pr', 'Trend in annual precipitation (abrupt-4xCO2)')  

In [ ]:
plot_map_trend(pr_4xco2_trend_NorESM_y, 'mk_trend', 'Arctic annual trend in precipitation (abrupt-4xCO2)')

In [ ]:
plot_map_trend(pr_piControl_trend_NorESM_y, 'mk_trend', 'Arctic rend in annual precipitation (piControl)')

### Making a correlation matrix

- find latidue above 80 degrees
- flatten variables (with array.reshape[-1,1])
- make a dataframe with each variabel in the columns
- plot the correlation matrix.

In [28]:
# new ds with lat>80 degrees

max_lat = 90
min_lat = 70
max_lon = 178
min_lon = 0

mask_lon = (ds_4xco2_NorESM_y.lon >= min_lon) & (ds_4xco2_NorESM_y.lon <= max_lon)
mask_lat = (ds_4xco2_NorESM_y.lat >= min_lat) & (ds_4xco2_NorESM_y.lat<= max_lat)

ds_arctic_4xco2 = ds_all.where(mask_lon & mask_lat, drop=True)
ds_arctic_4xco2

<xarray.Dataset>
Dimensions:  (time: 360, lat: 22, lon: 143, lev: 32)
Coordinates:
  * time     (time) object 0001-01-16 12:00:00 ... 0030-12-16 12:00:00
  * lat      (lat) float64 70.21 71.15 72.09 73.04 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 173.8 175.0 176.2 177.5
  * lev      (lev) float64 0.9926 0.9763 0.9575 ... 0.01436 0.007595 0.003643
Data variables: (12/16)
    pr       (time, lat, lon) float32 dask.array<chunksize=(120, 22, 143), meta=np.ndarray>
    ccn      (time, lat, lon) float32 dask.array<chunksize=(120, 22, 143), meta=np.ndarray>
    cdnc     (time, lev, lat, lon) float32 dask.array<chunksize=(120, 32, 22, 48), meta=np.ndarray>
    co2      (time, lev, lat, lon) float32 dask.array<chunksize=(120, 32, 22, 48), meta=np.ndarray>
    emibc    (time, lat, lon) float32 dask.array<chunksize=(120, 22, 143), meta=np.ndarray>
    emibvoc  (time, lat, lon) float32 dask.array<chunksize=(120, 22, 143), meta=np.ndarray>
    ...       ...
    emiso4   (time, lat, lon) float32 dask.array<chunksize=(120, 22, 143), meta=np.ndarray>
    emiss    (time, lat, lon) float32 dask.array<chunksize=(120, 22, 143), meta=np.ndarray>
    emivoc   (time, lat, lon) float32 dask.array<chunksize=(120, 22, 143), meta=np.ndarray>
    prc      (time, lat, lon) float32 dask.array<chunksize=(120, 22, 143), meta=np.ndarray>
    so2      (time, lev, lat, lon) float32 dask.array<chunksize=(120, 32, 22, 48), meta=np.ndarray>
    ts       (time, lat, lon) float32 dask.array<chunksize=(120, 22, 143), meta=np.ndarray>

In [42]:
#arrays = [ds_arctic_4xco2[i].values.flatten() for i in var_list] 


In [43]:
var_arrays = []
for var in var_list:
    var_arrays.append(np.array([ds_arctic_4xco2[var]]).reshape(-1,1))
    
    
#ar = np.array([ds_arctic_4xco2['pr']])
#ar.reshape(-1,1)
var_arrays[0]



array([[3.3488475e-05],
       [3.4001339e-05],
       [3.3211356e-05],
       ...,
       [5.4852994e-06],
       [5.4848915e-06],
       [5.4845164e-06]], dtype=float32)

In [44]:
var_arrays

[array([[3.3488475e-05],
        [3.4001339e-05],
        [3.3211356e-05],
        ...,
        [5.4852994e-06],
        [5.4848915e-06],
        [5.4845164e-06]], dtype=float32),
 array([[45254152.],
        [46940708.],
        [48607456.],
        ...,
        [11087151.],
        [11088034.],
        [11089019.]], dtype=float32),
 array([[1.5626485e+08],
        [1.6654659e+08],
        [1.6729904e+08],
        ...,
        [          nan],
        [          nan],
        [          nan]], dtype=float32),
 array([[0.00042958],
        [0.00042957],
        [0.00042957],
        ...,
        [0.00042902],
        [0.00042902],
        [0.00042902]], dtype=float32),
 array([[3.5773120e-17],
        [5.1182684e-17],
        [9.8632255e-17],
        ...,
        [3.7612455e-17],
        [3.7612455e-17],
        [3.7612455e-17]], dtype=float32),
 array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]], dtype=float32),
 array([[0.],
        [0.],


In [45]:
data = {str(var_list[0]): var_arrays[0], str(var_list[1]): var_arrays[1]}  

In [95]:
len(var_arrays[0])

1132560

In [92]:
var_arrays[0][0][0]

3.3488475e-05

In [96]:
l=[]
l.append(data['pr'][0][0])
l.append(data['pr'][1][0])

for i in range(len(var_arrays)):
    for j in range(len(var_arrays[0])):
        if len(var_arrays[i][j])==1:
            var_arrays[i][j]=var_arrays[i][j][0]
    

In [ ]:
var_arrays

In [83]:
df_var = pd.DataFrame(var_arrays[:][:][0], var_list)
df_var

ValueError: Shape of passed values is (1132560, 1), indices imply (16, 1)

In [65]:
df_var.corr()

""


### Time series plots - Global and Arctic mean

In [ ]:
def weighted_mean(ds):
    weights = np.cos(np.deg2rad(ds.lat))
    weights.name = "weights"
    air_weighted = ds.weighted(weights)
    weighted_mean = air_weighted.mean(("lon", "lat"))
    return weighted_mean


In [ ]:
# Arctic 

max_lat = 90
min_lat = 70
max_lon = 135
min_lon = 45

mask_lon = (ds_4xco2_NorESM_y.lon >= min_lon) & (ds_4xco2_NorESM_y.lon <= max_lon)
mask_lat = (ds_4xco2_NorESM_y.lat >= min_lat) & (ds_4xco2_NorESM_y.lat<= max_lat)

ds_arctic_4xco2 = ds_4xco2_NorESM_y.where(mask_lon & mask_lat, drop=True)



In [ ]:
max_lat = 90
min_lat = 70
max_lon = 135
min_lon = 45

mask_lon_pi = (ds_piControl_NorESM_y.lon >= min_lon) & (ds_piControl_NorESM_y.lon <= max_lon)
mask_lat_pi = (ds_piControl_NorESM_y.lat >= min_lat) & (ds_piControl_NorESM_y.lat<= max_lat)


ds_arctic_piControl = ds_piControl_NorESM_y.where(mask_lon & mask_lat, drop=True)

In [ ]:
#ds_arctic_4xco2_y = annual(ds_arctic_4xco2)
#ds_arctic_4xco2_y = fix_units(ds_arctic_4xco2_y )

#ds_arctic_piControl_y = annual(ds_arctic_piControl)
#ds_arctic_piControl_y = fix_units(ds_arctic_piControl_y )

In [ ]:
arctic_mean_4xco2 = weighted_mean(ds_arctic_4xco2_y)
global_mean_4xco2 = weighted_mean(ds_4xco2_NorESM_y)


arctic_mean_piControl = weighted_mean(ds_arctic_piControl_y)
global_mean_piControl = weighted_mean(ds_piControl_NorESM_y)

In [ ]:
#arctic_mean_piControl['pr'] = arctic_mean_piControl['pr']/86400 # mm/year
#ds['pr'] = ds['pr'].assign_attrs(units='mm/year')
#arctic_mean_piControl['ts'] = arctic_mean_piControl['ts']+273.15 # C

In [ ]:
arctic_mean_piControl.pr.values

In [ ]:
#air_weighted = ds_4xco2_NorESM_y.weighted(weights)
#air_weighted

In [ ]:
#weighted_mean_4xco2 = air_weighted.mean(("lon", "lat"))
#weighted_mean_4xco2

In [ ]:
def fancy(ax, fontsize):
    
    # thickning the axes spines
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(2)
        ax.spines[axis].set_color('k')
        
    # set the fontsize for all your ticks
    #fontsize = 15
    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
        
    # properties of the ticks
    ax.tick_params(direction='out', length=8, width=2, pad=10, bottom=True, top=False, left=True, right=False, color='k')
    
    # add a grid to the plot
    ax.grid(True, alpha=0.5)
    
    # mask top and right spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

In [ ]:
timepoints = []

for i in range(len(global_mean_4xco2['time'])):
    timepoints.append(i)

timepoints
timepoints2 = []
for i in range(len(global_mean_piControl['time'])):
    timepoints2.append(i)

In [ ]:
import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(10, 5))

gs = gridspec.GridSpec(ncols=2, nrows=3, hspace = 0.7, wspace = 0.3, top = 1,
                       bottom = 0, left = 0, right = 1)

# create an ax with gs
ax = fig.add_subplot(gs[0, 0])
ax.plot(timepoints, global_mean_4xco2['ts'], label='4xco2')
ax.plot(timepoints, global_mean_piControl['ts'], label='piControl')
ax.set_title('Global mean', size=15)
ax.set_ylabel('Temperature [C$^\circ$]')
ax.legend()
fancy(ax, 10)
#fig.suptitle('h')
#hide_ticks(ax)
#ax.text(0, 0, 'gs[0:4, 0:2]', fontsize=17, color='red', transform=ax.transAxes, 
        #horizontalalignment='center', verticalalignment='center')

ax = fig.add_subplot(gs[1, 0])
#hide_ticks(ax)
ax.plot(timepoints, global_mean_4xco2['pr'], label='4xco2')
ax.plot(timepoints, global_mean_piControl['pr'], label='piControl')
#ax.set_title('Precipitation')
ax.set_ylabel('Precipitation [mm]')
ax.legend()
fancy(ax, 10)

ax = fig.add_subplot(gs[2, 0])
#hide_ticks(ax)
ax.plot(timepoints, global_mean_4xco2['ccn'], label='4xco2')
ax.plot(timepoints, global_mean_piControl['ccn'], label='piControl')
#ax.set_title('CCN')
ax.set_xlabel('Time [years]')
ax.set_ylabel('CCN [m$^{-3}$]')
ax.legend()
fancy(ax, 10)

ax = fig.add_subplot(gs[0, 1])
#hide_ticks(ax)
ax.plot(timepoints, arctic_mean_4xco2['ts'], label='4xco2')
ax.plot(timepoints, arctic_mean_piControl['ts'], label='piControl')
ax.set_title('Arctic mean', size=15)
ax.legend()
fancy(ax, 10)

ax = fig.add_subplot(gs[1, 1])
#hide_ticks(ax)
ax.plot(timepoints, arctic_mean_4xco2['pr'], label='4xco2')
ax.plot(timepoints, arctic_mean_piControl['pr'], label='piControl')
#ax.set_title('Precipitation')
ax.legend()
fancy(ax, 10)

ax = fig.add_subplot(gs[2, 1])
#hide_ticks(ax)
ax.plot(timepoints, arctic_mean_4xco2['ccn'], label='4xco2')
ax.plot(timepoints, arctic_mean_piControl['ccn'], label='piControl')
#ax.set_title('CCN')
ax.set_xlabel('Time [years]')
ax.legend()
fancy(ax, 10)


